In [16]:
import pandas as pd
import re

pos_df = pd.read_csv("pos_topic_ner_sentiment_embedding_features.csv")
outfit_df = pd.read_csv("outfit.csv")

In [17]:
pos_df.head()

,id,date,article_title,article_text,author,places,topics,n_places,clean_text,region,...,ner_loc,polarity,subjectivity,embedding_vector,pca_x,pca_y,tsne_x,tsne_y,umap_x,umap_y
0,6072,17-MAR-1987 12:09:38.91,S/P MAY DOWNGRADE CHI-CHI'S <CHIC> CONVERTIBLES,"NEW YORK, March 17 -\n\nStandard and Poor's Co...",NaN,usa,NaN,1,"NEW YORK, March 17 -\n\nStandard and Poor's Co...",us,...,0,-0.078864,0.462955,"[-0.05211601033806801, -0.11056090146303177, 0...",0.171572,0.029933,25.270853,13.065409,8.243795,11.944348
1,21069,19-OCT-1987 14:40:54.90,<THERMASCAN INC> RECEIVES FDA APPROVAL ON DRUG,"NEW YORK, Oct 19 -\n\nThermascan Inc said the ...",NaN,usa,NaN,1,"NEW YORK, Oct 19 -\n\nThermascan Inc said the ...",us,...,0,0.085950,0.292562,"[-0.016178948804736137, 0.0008432294125668705,...",0.034821,0.182132,39.910797,24.604310,5.767273,13.556754
2,184,26-FEB-1987 18:12:10.87,AMERICAN TRAVELLERS <ATVC> EXPANDS OPERATIONS,"WARRINGTON, Pa., Feb 26 -\n\nAmerican Travelle...",NaN,usa,NaN,1,"WARRINGTON, Pa., Feb 26 -\n\nAmerican Travelle...",us,...,0,0.000000,0.000000,"[0.04712970182299614, -0.03134236857295036, -0...",0.042796,0.373436,23.789541,-4.021205,6.800491,12.325949
3,14549,7-APR-1987 15:55:08.94,PACIFIC LIGHTING CORP <PLT> QTLY DIVIDEND,"LOS ANGELES, April 7 -\n\nShr 87 cts vs 87 cts...",NaN,usa,earn,1,"LOS ANGELES, April 7 -\n\nShr 87 cts vs 87 cts...",us,...,0,0.000000,0.000000,"[-0.1191648617386818, 0.06715720146894455, -0....",0.442399,-0.090178,89.047264,-8.742058,13.048009,11.207725
4,19286,19-JUN-1987 13:16:09.45,N.Y. TIMES <NYT> TO BUY GWINNETT DAILY NEWS,"NEW YORK, June 19 -\n\nThe New York Times Co s...",NaN,usa,acq,1,"NEW YORK, June 19 -\n\nThe New York Times Co s...",us,...,0,0.056818,0.347727,"[-0.0031330108176916838, 0.009149694815278053,...",0.022463,0.093347,-13.378264,42.326157,2.839716,13.868253


In [18]:
outfit_df.head()

,Path,Class,Total number of words,Word diversity,FFT mean,FFT median,FFT stddev,FFT min,FFT max,FFT histogram bin 1,...,POS FFT mean,POS FFT median,POS FFT stddev,POS FFT min,POS FFT max,POS FFT histogram bin 1,POS FFT histogram bin 2,POS FFT histogram bin 3,POS FFT histogram bin 4,POS FFT histogram bin 5
0,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,europe,209,0.545455,232.693,50.2142,631.051,0.0,8387.0,0.990431,...,0.009756,0.004878,0.009756,0.004878,34.693200,6.362240,91.150600,0.00000,1233.000,0.995122
1,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,europe,109,0.724771,137.097,68.6545,377.580,0.0,3790.0,0.981651,...,0.009259,0.018519,0.009259,0.018519,0.009259,31.418200,9.000000,74.38680,1.000,727.000000
2,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,europe,133,0.609023,0.000,38.4787,NaN,0.0,0.0,0.977444,...,0.007692,0.015385,0.007692,0.007692,0.007692,0.007692,29.611900,5.31428,76.169,0.000000
3,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,europe,191,0.633508,0.000,210.3530,NaN,0.0,0.0,0.994764,...,0.038043,0.016304,0.005435,0.010870,0.010870,NaN,67.108400,NaN,0.000,0.000000
4,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,europe,102,0.607843,NaN,98.8447,NaN,0.0,0.0,0.931373,...,0.010417,22.745600,8.339280,66.459800,0.000000,635.000000,0.989583,0.00000,0.000,0.000000


In [19]:
def extract_id_from_path(path):
    if not isinstance(path, str):
        return None
    # Extract final digits before extension
    match = re.search(r'(\d+)\.txt$', path)
    return int(match.group(1)) if match else None

outfit_df["id"] = outfit_df["Path"].apply(extract_id_from_path)


In [20]:
outfit_df[["Path", "id"]].head()


,Path,id
0,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,10013
1,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,10015
2,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,10016
3,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,1005
4,/Users/tiruskimani/MSDATA/cis733/reuters/artic...,1008


In [21]:
merged = pos_df.merge(outfit_df, on="id", how="inner")


In [22]:
print("Rows in df1:", len(pos_df))
print("Rows in df2:", len(outfit_df))
print("Rows in merged:", len(merged))

print("Missing matches:", sum(outfit_df["id"].isna()))

Rows in df1: 4072
Rows in df2: 4072
Rows in merged: 4072
Missing matches: 0


In [23]:
merged_inner = pos_df.merge(outfit_df, on="id", how="left")


In [24]:
merged_inner.to_csv("merged_pos_outfit.csv", index=False)

In [14]:
print("Rows in df1:", len(pos_df))
print("Rows in df2:", len(outfit_df))
print("Rows in merged:", len(merged_inner))

print("Missing matches:", sum(outfit_df["id"].isna()))

Rows in df1: 4072
Rows in df2: 4072
Rows in merged: 4072
Missing matches: 0


In [25]:
merged_inner.head()

,id,date,article_title,article_text,author,places,topics,n_places,clean_text,region,...,POS FFT mean,POS FFT median,POS FFT stddev,POS FFT min,POS FFT max,POS FFT histogram bin 1,POS FFT histogram bin 2,POS FFT histogram bin 3,POS FFT histogram bin 4,POS FFT histogram bin 5
0,6072,17-MAR-1987 12:09:38.91,S/P MAY DOWNGRADE CHI-CHI'S <CHIC> CONVERTIBLES,"NEW YORK, March 17 -\n\nStandard and Poor's Co...",NaN,usa,NaN,1,"NEW YORK, March 17 -\n\nStandard and Poor's Co...",us,...,0.009346,24.774100,7.514610,68.565500,0.000000,686.000000,0.990654,0.000000,0.000000,0.0
1,21069,19-OCT-1987 14:40:54.90,<THERMASCAN INC> RECEIVES FDA APPROVAL ON DRUG,"NEW YORK, Oct 19 -\n\nThermascan Inc said the ...",NaN,usa,NaN,1,"NEW YORK, Oct 19 -\n\nThermascan Inc said the ...",us,...,NaN,0.000000,0.000000,0.947917,0.000000,0.000000,0.000000,0.052083,0.000000,0.0
2,184,26-FEB-1987 18:12:10.87,AMERICAN TRAVELLERS <ATVC> EXPANDS OPERATIONS,"WARRINGTON, Pa., Feb 26 -\n\nAmerican Travelle...",NaN,usa,NaN,1,"WARRINGTON, Pa., Feb 26 -\n\nAmerican Travelle...",us,...,0.000000,153.000000,0.942857,0.028571,0.000000,0.000000,0.028571,0.000000,0.000000,0.0
3,14549,7-APR-1987 15:55:08.94,PACIFIC LIGHTING CORP <PLT> QTLY DIVIDEND,"LOS ANGELES, April 7 -\n\nShr 87 cts vs 87 cts...",NaN,usa,earn,1,"LOS ANGELES, April 7 -\n\nShr 87 cts vs 87 cts...",us,...,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,19286,19-JUN-1987 13:16:09.45,N.Y. TIMES <NYT> TO BUY GWINNETT DAILY NEWS,"NEW YORK, June 19 -\n\nThe New York Times Co s...",NaN,usa,acq,1,"NEW YORK, June 19 -\n\nThe New York Times Co s...",us,...,2.956470,46.420800,1.000000,403.000000,0.967033,0.021978,0.000000,0.000000,0.010989,0.0
